In [1]:
!pip install transformers
!pip install huggingface_hub
!pip install --upgrade datasets fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification , Trainer, TrainingArguments

In [3]:
from datasets import load_dataset
dataset = load_dataset('imdb')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
train = dataset['train'].select(range(1000))
test = dataset['test'].select(range(500))

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
train

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [7]:
# Tokenization function
def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

In [8]:
# Apply tokenization + rename + format in a single flow
def preprocess(ds):
    ds = ds.map(tokenize_fn, batched=True, remove_columns=["text"])  # remove raw text (saves memory)
    ds = ds.rename_column("label", "labels")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds


In [9]:
train_f = preprocess(train)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)

In [12]:
test_f = preprocess(test)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./bert-finetuned-imdb",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"
)

In [14]:

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_f,
    eval_dataset=test_f
)

In [15]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=125, training_loss=0.000776439905166626, metrics={'train_runtime': 34.4841, 'train_samples_per_second': 28.999, 'train_steps_per_second': 3.625, 'total_flos': 66233699328000.0, 'train_loss': 0.000776439905166626, 'epoch': 1.0})

In [16]:
trainer.save_model('finetune')
tokenizer.save_pretrained('finetune')


('finetune/tokenizer_config.json',
 'finetune/special_tokens_map.json',
 'finetune/vocab.txt',
 'finetune/added_tokens.json',
 'finetune/tokenizer.json')

In [17]:
metrics = trainer.evaluate()

In [18]:
print(metrics)

{'eval_loss': 3.303499033791013e-05, 'eval_runtime': 4.2303, 'eval_samples_per_second': 118.195, 'eval_steps_per_second': 14.893, 'epoch': 1.0}


In [19]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained('finetune')
tokenizer = AutoTokenizer.from_pretrained('finetune')


In [20]:
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
result = classifier("the dress is great")
print(result)


Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9997984766960144}]


In [24]:
text = input()

the movie is bad


In [25]:
result = classifier(text)

In [26]:
result

[{'label': 'NEGATIVE', 'score': 0.9999662637710571}]